# Radiate Traces

In [1]:
import os

# Import GDS modules
from lifelike_gds.arango_network.radiate_trace import RadiateTrace
from lifelike_gds.arango_network.reactome import *
from lifelike_gds.arango_network.reactome_notebook_functions import *

# Ignore warnings
import warnings

warnings.filterwarnings('ignore')


## Settings
Make sure to change to the correct one based on your local settings

In [2]:
# Directory where to look for input data
input_dir = './input'

# Directory where to output results
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

# Reactome arango database connection parameters
# If you are running this notebook from Lifelike's online training BinderHub website,
# then these paremeters are already set for you in the environment.
arango_dbname = os.getenv('REACTOME_ARANGO_DATABASE', 'reactome')

## Parameters
Set parameters
- source_name: name for the source entities. 
- source_file: source file name in input dir


In [3]:
source_name = 'ph-updown-metabolites'

nodes_select_file = f'Radiate_analysis_for_{source_name}.xlsx'

## Connect to arango database

In [4]:
database = ReactomeDB(arango_dbname)

## Load graph from arango graph database to memery

In [5]:
tracegraph = RadiateTrace(Reactome(database))
# set up output directory where the excel and graph files will write to
tracegraph.datadir = output_dir
# initiate tracegraph by loading graph data from arango
# a networkx graph is created here.  
tracegraph.init_default_graph()

INFO: load reactome graph
INFO: MultiDirectedGraph with 71225 nodes and 112575 edges


## Get source nodes, run radiate analysis, and export data to excel

In [6]:
ids_ph = [
    'R-ALL-83957', # Hypoxanthine (extracellular region)
    'R-ALL-3296298', # x12(S)-HETE (extracellular region)
    'R-ALL-210382',	# Glutamic acid (extracellular region)
    'R-ALL-419305',	# Sphingosine 1 phosphate (extracellular region)
    'R-ALL-351977', # Arginine (extracellular region)
    'R-ALL-351970'	# Histidine (extracellular region)
]
source_nodes = database.get_nodes_by_attr(ids_ph, 'stId')
selected_forward_nodes = get_selected_nodes(input_dir, nodes_select_file, 'pageranks')
selected_reverse_nodes = get_selected_nodes(input_dir, nodes_select_file, 'reverse pageranks')

FileNotFoundError: [Errno 2] No such file or directory: './input/Radiate_analysis_for_ph-updown-metabolites.xlsx'

In [ ]:
export_radiate_traces(database, tracegraph, source_name, source_nodes, selected_forward_nodes, None)
export_radiate_traces(database, tracegraph, source_name, source_nodes, None, selected_reverse_nodes)
